In [0]:
dbutils.fs.mount(
  source = "wasbs://bndl-data@bndldata01.blob.core.windows.net",
  mount_point = "/mnt/bndldata",
  extra_configs = {"fs.azure.account.key.bndldata01.blob.core.windows.net":AccountKey})


True

In [0]:
AccountKey = "NDJ9/LmL0rSSKMXR4F30mH4DuR86NWAwrwM+X0nLnV0bC1l3fYUpugQDCslsvR6FuRVg+NFnH8eZ+AStafDOSg=="

In [0]:
display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/mnt/bndldata,wasbs://bndl-data@bndldata01.blob.core.windows.net,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
/,DatabricksRoot,
/volume,DbfsReserved,


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, max, min, when, desc, asc
from pyspark.sql import functions as F

In [0]:
buildings = spark.read.format("csv").option("header","true").load("/mnt/bndldata/raw-data/buildings.csv")
leases = spark.read.format("csv").option("header","true").load("/mnt/bndldata/raw-data/leases.csv")

In [0]:
leases.head(2)

[Row(Location Code='AK0009', Real Property Asset Name='POST OFFICE AND CU H', Installation Name='NA', Federal Leased Code='L', GSA Region='10', Street Address='2ND ST', City='WRANGELL', State='AK', Zip Code='99929', Latitude='56.473949', Longitude='-132.38812', Building Rentable Square Feet='97', Available Square Feet='0', Congressional District='0200', Congressional District Representative='MARY PELTOLA', Lease Number='LAK00348', Lease Effective Date='2022-10-01', Lease Expiration Date='2027-09-30', Real Property Asset type='BUILDING'),
 Row(Location Code='AK3139', Real Property Asset Name='TROUTTE CENTER', Installation Name='NA', Federal Leased Code='L', GSA Region='10', Street Address='329 HARBOR DR', City='SITKA', State='AK', Zip Code='99835', Latitude='57.0502849', Longitude='-135.33364', Building Rentable Square Feet='951', Available Square Feet='0', Congressional District='0200', Congressional District Representative='MARY PELTOLA', Lease Number='LAK07113', Lease Effective Date=

In [0]:
buildings.printSchema()

root
 |-- Location Code: string (nullable = true)
 |-- Real Property Asset Name: string (nullable = true)
 |-- Installation Name: string (nullable = true)
 |-- Owned or Leased: string (nullable = true)
 |-- GSA Region: string (nullable = true)
 |-- Street Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zip Code: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Building Rentable Square Feet: string (nullable = true)
 |-- Available Square Feet: string (nullable = true)
 |-- Construction Date: string (nullable = true)
 |-- Congressional District: string (nullable = true)
 |-- Congressional District Representative Name: string (nullable = true)
 |-- Building Status: string (nullable = true)
 |-- Real Property Asset Type: string (nullable = true)



In [0]:
leases.printSchema()

root
 |-- Location Code: string (nullable = true)
 |-- Real Property Asset Name: string (nullable = true)
 |-- Installation Name: string (nullable = true)
 |-- Federal Leased Code: string (nullable = true)
 |-- GSA Region: string (nullable = true)
 |-- Street Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zip Code: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Building Rentable Square Feet: string (nullable = true)
 |-- Available Square Feet: string (nullable = true)
 |-- Congressional District: string (nullable = true)
 |-- Congressional District Representative: string (nullable = true)
 |-- Lease Number: string (nullable = true)
 |-- Lease Effective Date: string (nullable = true)
 |-- Lease Expiration Date: string (nullable = true)
 |-- Real Property Asset type: string (nullable = true)



In [0]:
# Total Rentable Square Feet by State
total_rentable_sqft_by_state = buildings.groupBy("State").agg(
    F.sum(F.col("Building Rentable Square Feet").cast("float")).alias("Total Rentable Square Feet")
)
total_rentable_sqft_by_state.orderBy(desc("Total Rentable Square Feet")).collect()

[Row(State='DC', Total Rentable Square Feet=53849974.945503235),
 Row(State='MD', Total Rentable Square Feet=28984288.314453125),
 Row(State='CA', Total Rentable Square Feet=28335853.532218933),
 Row(State='VA', Total Rentable Square Feet=25353905.739440918),
 Row(State='TX', Total Rentable Square Feet=23291260.398838043),
 Row(State='NY', Total Rentable Square Feet=17801643.341400146),
 Row(State='MO', Total Rentable Square Feet=16110110.77960205),
 Row(State='FL', Total Rentable Square Feet=12233671.1171875),
 Row(State='PA', Total Rentable Square Feet=11397751.472167969),
 Row(State='CO', Total Rentable Square Feet=10878489.513671875),
 Row(State='GA', Total Rentable Square Feet=10420422.404785156),
 Row(State='IL', Total Rentable Square Feet=9703312.453491211),
 Row(State='WA', Total Rentable Square Feet=7777220.030426025),
 Row(State='OH', Total Rentable Square Feet=7522578.959350586),
 Row(State='MI', Total Rentable Square Feet=6189479.265609741),
 Row(State='MA', Total Rentable 

In [0]:
    # Number of Buildings by Ownership Type
    building_count_by_ownership = buildings.groupBy("Owned or Leased").agg(
        F.count("*").alias("Building Count")) 
    building_count_by_ownership.show()

+---------------+--------------+
|Owned or Leased|Building Count|
+---------------+--------------+
|              F|          1807|
|              L|          6805|
+---------------+--------------+



In [0]:
# Total Leased Square Feet by Region
total_leased_sqft_by_region = leases.groupBy("GSA Region").agg(
    F.sum(F.col("Building Rentable Square Feet").cast("float")).alias("Total Leased Square Feet")
)
total_leased_sqft_by_region.orderBy(desc("Total Leased Square Feet")).show(5)

+----------+------------------------+
|GSA Region|Total Leased Square Feet|
+----------+------------------------+
|        11|      6.43012047734375E7|
|         4|    2.9586947479492188E7|
|         3|    2.5367902896453857E7|
|         7|    2.4768737575927734E7|
|         9|    2.4623457528320312E7|
+----------+------------------------+
only showing top 5 rows



In [0]:
# Join the dataframes on common columns
combined_df = buildings.join(leases, on=["Location Code"], how="inner")


In [0]:
# Count of Leases by Expiration Year
leases_by_expiration_year = leases.withColumn(
    "Lease Expiration Year", F.year("Lease Expiration Date")
).groupBy("Lease Expiration Year").agg(
    F.count("*").alias("Lease Count")
)
leases_by_expiration_year.orderBy(desc("Lease Count")).show(5)

+---------------------+-----------+
|Lease Expiration Year|Lease Count|
+---------------------+-----------+
|                 2026|        843|
|                 2027|        805|
|                 2025|        787|
|                 2028|        710|
|                 2024|        629|
+---------------------+-----------+
only showing top 5 rows



In [0]:
# Maximum Lease Duration by Region
lease_duration_by_region = leases.withColumn(
    "Lease Duration", 
    F.datediff(F.col("Lease Expiration Date"), F.col("Lease Effective Date"))
).groupBy("GSA Region").agg(
    F.max("Lease Duration").alias("Max Lease Duration (days)")
)
lease_duration_by_region.orderBy(desc("Max Lease Duration (days)")).show(5)

+----------+-------------------------+
|GSA Region|Max Lease Duration (days)|
+----------+-------------------------+
|         5|                    31945|
|         2|                    22654|
|         7|                    16070|
|         3|                    14945|
|         4|                    14609|
+----------+-------------------------+
only showing top 5 rows



In [0]:
# Categorize leases based on duration
leases_with_duration_category = leases.withColumn(
    "Lease Duration (days)",
    F.datediff(F.col("Lease Expiration Date"), F.col("Lease Effective Date"))
).withColumn(
    "Lease Duration Category", F.when(F.col("Lease Duration (days)") <= 365, F.lit("Short-term"))
    .when((F.col("Lease Duration (days)") > 365) & (F.col("Lease Duration (days)") <= 365*5), F.lit("Medium-term"))
    .otherwise(F.lit("Long-term")))
leases_with_duration_category.select("Real Property Asset Name","Building Rentable Square Feet","Lease Number","Lease Duration (days)","Lease Duration Category").show(5)

+------------------------+-----------------------------+------------+---------------------+-----------------------+
|Real Property Asset Name|Building Rentable Square Feet|Lease Number|Lease Duration (days)|Lease Duration Category|
+------------------------+-----------------------------+------------+---------------------+-----------------------+
|    POST OFFICE AND CU H|                           97|    LAK00348|                 1825|            Medium-term|
|          TROUTTE CENTER|                          951|    LAK07113|                 7304|              Long-term|
|          PETERSON TOWER|                         5283|    LAK07230|                 6443|              Long-term|
|    TUDOR BUSINESS CNTER|                        89049|    LAK07436|                 7304|              Long-term|
|        MICHAEL BUILDING|                        33855|    LAK06502|                 9131|              Long-term|
+------------------------+-----------------------------+------------+---

In [0]:
#Add a column indicating if the lease is active or expired
leases_with_status = leases.withColumn(
    "Lease Status", 
    F.when(F.col("Lease Expiration Date") > F.current_date(), F.lit("Active"))
    .otherwise(F.lit("Expired"))
)
leases_with_status.select("Real Property Asset Name","Building Rentable Square Feet","Lease Number","Lease Status").orderBy(desc("Building Rentable Square Feet")).show(5)

+------------------------+-----------------------------+------------+------------+
|Real Property Asset Name|Building Rentable Square Feet|Lease Number|Lease Status|
+------------------------+-----------------------------+------------+------------+
|    UNIVERSITY PLACE ...|                         9999|    LAL60393|      Active|
|       HALLMARK BUILDING|                         9999|    LVA12693|      Active|
|    SPRINGFIELD OH (1...|                         9999|    LOH01170|      Active|
|    MUSKEGON MI (340 ...|                         9999|    LMI00955|      Active|
|    1680 LAKE FRONT C...|                         9999|    LTX17415|      Active|
+------------------------+-----------------------------+------------+------------+
only showing top 5 rows



In [0]:
leases_with_status.repartition(3).write.mode("overwrite").option("header","true").csv("/mnt/bndldata/transformed-data/leases_with_status.csv")
leases_with_duration_category.repartition(3).write.mode("overwrite").option("header","true").csv("/mnt/bndldata/transformed-data/leases_with_duration_category.csv")
lease_duration_by_region.write.mode("overwrite").option("header","true").csv("/mnt/bndldata/transformed-data/lease_duration_by_region.csv")
leases_by_expiration_year.write.mode("overwrite").option("header","true").csv("/mnt/bndldata/transformed-data/leases_by_expiration_year.csv")
total_leased_sqft_by_region.write.mode("overwrite").option("header","true").csv("/mnt/bndldata/transformed-data/total_leased_sqft_by_region.csv")